In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate




··········


In [ ]:


!mkdir -p drive
!google-drive-ocamlfuse drive



fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [ ]:
from  urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import pandas as pd
import  time
from urllib.request import urlretrieve
import cv2
import numpy as np
from urllib.parse import urlparse
import os
from urllib.error import HTTPError
import requests

# starttime = time.datetime()





In [ ]:
beginurl = "http://www.dermnet.com"
url = "http://www.dermnet.com/dermatology-pictures-skin-disease-pictures"
save_category_list = []
fullcategory = []
subfullcategory = []
save_image_link = []



parse = urlopen(url)
soup = bs(parse, 'html.parser')
findall_a_in_soup = soup.find('table').find_all('a')

for i in findall_a_in_soup:
    remove_href = i.get('href')
    save_category_list.append(remove_href)
    # print(category)
    link_category = beginurl + remove_href
#     print(link_category)
    paser_subcategory = urlopen(link_category)
    soup_subcategory = bs(paser_subcategory, 'html.parser')
    findall_a_in_soup_subcategory = soup_subcategory.find('table').find_all('a')
    
    
    for link_id in findall_a_in_soup_subcategory:
      remove_href_in_link_id = link_id.get('href')
      link_id_category = beginurl + remove_href_in_link_id
      parse_image_link = urlopen(link_id_category)
      soup_parse_image_link = bs(parse_image_link, 'html.parser')
      soupimagefinddiv = soup_parse_image_link.findAll('div', 'thumbnails')#fuck you
      #print first page before pagination
      if soup_parse_image_link.find('div', 'pagination'):
        a_in_pagination = soup_parse_image_link.find('div', 'pagination').findAll('a')
        a_in_pagination.pop(-1)
        
        for a in a_in_pagination:
          remove_href_from_a = a.get('href')
          pagin_link = beginurl + remove_href_from_a
          parse_pagin = urlopen(pagin_link)
          soup_parse_pagin = bs(parse_pagin)
          page_soup_image_findiv = soup_parse_pagin.findAll('div', 'thumbnails')
          
          for in_page_soup_image_findiv in page_soup_image_findiv:
            get_image_in_page_soup_image_findiv = in_page_soup_image_findiv.find('img')
#             print(get_image_in_page_soup_image_findiv)
            image_in_page_soup_image_findiv = get_image_in_page_soup_image_findiv.get('src')
            save_image_link.append(image_in_page_soup_image_findiv)
#             fullcategory.append(remove_href_in_link_id)
            fullcategory.append(remove_href)
            subfullcategory.append(remove_href_from_a)
            

    


In [ ]:
data = pd.DataFrame({"category" : save_category_list})
data.to_csv("drive/imageclassi/category.csv", index= False)


In [ ]:

save_fullcategory = pd.DataFrame({"category" : fullcategory})
save_fullcategory.to_csv("drive/imageclassi/fullcategory.csv", index= False)


In [ ]:

save_subfullcategory = pd.DataFrame({"category" : subfullcategory})
save_subfullcategory.to_csv("drive/imageclassi/subfullcategory.csv", index= False)


In [ ]:
save_file = pd.DataFrame({'category': fullcategory, 'subcategory':subfullcategory})
save_file.to_csv('drive/imageclassi/dermnet.csv', index=False)

In [ ]:
save_file = pd.DataFrame({'category': fullcategory, 'subcategory':subfullcategory, 'imagelink' :save_image_link})
save_file.to_csv('drive/imageclassi/dermnetwithlink.csv', index=False)

In [ ]:
for each_image_link in save_image_link[7500:]:
#   print(each_image_link)
  filename = each_image_link.split('/')[-1]
  resource = requests.get(each_image_link)
  if resource.status_code == 404:
    continue
  else:
    resource = urlopen(each_image_link)
    with open('drive/imageclassi/dermnet/'+ filename,  'wb') as localFile:
      localFile.write(resource.read())
 

In [ ]:
print(save_image_link[3])

http://www.dermnet.com/dn2/allJPGThumb3/acne-closed-comedo-20.jpg
